## **Introduction - Sentiment Analysis Part II**
So far we have tried:
- roberta pre trained
- zero shot learning
- Vader on translated tweets

Issue: none of them works well (not good classification and very imbalanced data). Potential solutions:
1. Fine-tune a model on domain-specific data: collect an Italian political sentiment dataset, fine-tune an Italian language model like AlBERTo, UmBERTo, or BERTino on this dataset. You can manually label a subset of your tweets (even ~1,000) and use active learning to scale up.
2. Use a weak supervision framework: if labeling is too expensive, use Snorkel or Skweak to create heuristic rules (e.g., hashtags, emojis, known party slogans). Combine weak labels into a probabilistic label model. Train a classifier on this noisy but large-scale data.
3. Refine your zero-shot prompt setup - If you use a zero-shot pipeline (e.g., HuggingFace pipeline("zero-shot-classification")), try:
    - Framing labels in political terms: instead of generic "positive/negative/neutral", try "supportive", "opposing", "neutral toward the politician" or their policies.
    - Use prompt engineering to improve performance: candidate_labels = ["support", "oppose", "neutral"], hypothesis_template = "This tweet expresses a {} opinion about politics."
4. Use sentiment + emotion + toxicity hybrid: often political tweets are more emotional than classically "positive" or "negative". Use an emotion classifier (anger, joy, fear, etc.). Combine with toxicity detection (like HateBERT, ToxicBERT, or Italian-specific versions).
Fuse signals into a sentiment interpretation model.

For sure, what we will do first is to consider only a filtered subset of the data, which should be the ones that are most expressive.

In [1]:
import torch
import pandas as pd
import numpy as np


## **Load the Data**

In [2]:
politicians_cleaned = pd.read_csv('politicians_data/politicians_classified.csv')

In [3]:
politicians_cleaned.head()

,Unnamed: 0,Date,ID,Content,Likes,politician,party,ideology_num,classification
0,1,2022-12-29 20:55:30+00:00,1608567407500754944,Congratulations to @netanyahu on the formation...,6021,Meloni,FdI,3,1
1,6,2022-12-27 18:00:00+00:00,1607798465198710784,Vogliamo restituire a questa Nazione l’ottimis...,14890,Meloni,FdI,3,1
2,7,2022-12-27 17:05:06+00:00,1607784649056727041,Cordiale conversazione telefonica con @Zelensk...,8589,Meloni,FdI,3,1
3,9,2022-12-25 00:08:43+00:00,1606804094102155264,Franco Frattini era un uomo garbato e intellig...,3260,Meloni,FdI,3,1
4,11,2022-12-24 07:18:01+00:00,1606549741168377856,Proteggere la libertà religiosa è un obiettivo...,5901,Meloni,FdI,3,1


In [4]:
data = politicians_cleaned.drop(columns=['Unnamed: 0', 'classification'])

## **Active Learning**
Active learning is a technique used to minimize labeling effort by selectively choosing which examples a human should label next.
How it works:
- Start with a small labeled set (e.g., 200 tweets).
- Train an initial model (using transfer learning).
- Use that model to predict on the unlabeled tweets.
- Select the most uncertain predictions (e.g., low confidence margin between classes).
- Manually label these tweets (they’re the most informative).
- Retrain the model with the new labeled data.
- Repeat 
Goal: Reach high model performance with far fewer labels than random sampling would require

SET OF LABELS:
1. Neutral / Informational: Factual updates, announcements, or descriptive statements with no clear emotional tone or stance.
2. Supportive / Affirmative / Celebratory: Positive endorsement or praise — for allies, institutions, values, events. Celebrating some achievement.
3. Attacking / Critical / Angry / Outraged: Expressing indignation, moral outrage, or rhetorical escalation. Direct attack, blame, or disapproval — usually aimed at political opponents.
4. Call to Action / Propaganda

### **Part 1 - Label 100 random tweets**

In [6]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load your tweets
df = data.sample(n=100, random_state=42).reset_index(drop=True)  

# Define tone labels
tone_labels = [
    "Neutral / Informational",
    "Supportive / Positive / Celebratory",
    "Critical / Angry",
    "Call to Action / Propaganda",
]

# Store dropdowns and layout
dropdowns = []
for i, row in df.iterrows():
    tweet_display = widgets.HTML(value=f"<b>Tweet {i+1}</b>: {row['Content']}")
    dropdown = widgets.Dropdown(
        options=[""] + tone_labels,
        description='Tone:',
        layout=widgets.Layout(width='70%')
    )
    dropdowns.append(dropdown)
    display(tweet_display, dropdown)

# Save button logic
def save_labels(_):
    for i, dropdown in enumerate(dropdowns):
        df.at[i, 'tone_label'] = dropdown.value
    df.to_csv("labeled_tweets_with_tone.csv", index=False)
    print("✅ Labels saved to labeled_tweets_with_tone.csv")

save_button = widgets.Button(description="💾 Save Labels", button_style='success')
save_button.on_click(save_labels)
display(save_button)


HTML(value='<b>Tweet 1</b>: Alla fine di questa avventura a me interesserà una sola cosa: sapere che abbiamo f…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 2</b>: #Salvini: Ho ricordato all’avvocato Conte che mettere sullo stesso piano USA e Cin…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 3</b>: Dobbiamo spingere sulla ripartenza. Dobbiamo correre veloci, facendo investimenti …

...

HTML(value='<b>Tweet 99</b>: Busta con proiettile alla consigliera della Lega in Campania, Carmela Rescigno, i…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 100</b>: Partecipare alla vita pubblica è un dovere. Pericle sosteneva“un uomo che non si…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

Button(button_style='success', description='💾 Save Labels', style=ButtonStyle())

### **Part 2 - Fine Tune Model on 100 labeled tweets**

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

# LOAD AND PREPROCESS LABELED DATA
df = pd.read_csv("labeled_tweets_with_tone.csv")
df = df.dropna(subset=["tone_label"])  # Remove unlabeled rows

# Label encoding
labels = df["tone_label"].unique().tolist()
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}
df["label_id"] = df["tone_label"].map(label2id)

# Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Content"].tolist(), df["label_id"].tolist(), test_size=0.2, random_state=42
)


In [20]:
# TOKENIZE
from transformers import AutoTokenizer

model_name = "Musixmatch/umberto-commoncrawl-cased-v1"  # You can swap for AlBERTo if preferred
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [21]:
import torch

# CONVERT TO PYTORCH
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)


In [22]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: False
GPU name: None


In [23]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

# Prepare data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)


# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(5):  
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Train loss: {total_loss / len(train_loader):.4f}")

    # Optional: evaluation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == batch["labels"]).sum().item()
            total += batch["labels"].size(0)
    print(f"Validation accuracy: {correct / total:.2%}")


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cpu
Epoch 1
Train loss: 1.5308
Validation accuracy: 22.50%
Epoch 2
Train loss: 1.4023
Validation accuracy: 37.50%
Epoch 3
Train loss: 1.0970
Validation accuracy: 40.00%
Epoch 4
Train loss: 0.7557
Validation accuracy: 40.00%
Epoch 5
Train loss: 0.4504
Validation accuracy: 40.00%


## 200 labels
Epoch 1
Train loss: 1.5308
Validation accuracy: 22.50%
Epoch 2
Train loss: 1.4023
Validation accuracy: 37.50%
Epoch 3
Train loss: 1.0970
Validation accuracy: 40.00%
Epoch 4
Train loss: 0.7557
Validation accuracy: 40.00%
Epoch 5
Train loss: 0.4504
Validation accuracy: 40.00%


### **Phase 3**

In [13]:
all_df = data

# Exclude already labeled ones (assuming tone_label exists only for labeled rows)
unlabeled_df = all_df[~all_df["Content"].isin(df["Content"])].reset_index(drop=True)

# Take a reasonable subset if the dataset is huge
texts_to_predict = unlabeled_df["Content"].tolist()


In [14]:
from torch.nn.functional import softmax

device = torch.device("cpu")
model.to(device)

# Make sure inputs are also on CPU
predict_encodings = tokenizer(texts_to_predict, truncation=True, padding=True, return_tensors="pt")
predict_encodings = {k: v.to(device) for k, v in predict_encodings.items()}

batch_size = 32
all_probs = []

model.eval()
with torch.no_grad():
    for i in range(0, len(texts_to_predict), batch_size):
        batch_texts = texts_to_predict[i:i+batch_size]
        batch_enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt")
        batch_enc = {k: v.to("cpu") for k, v in batch_enc.items()}  # or "mps" if you fix it
        outputs = model(**batch_enc)
        batch_probs = softmax(outputs.logits, dim=1).numpy()
        all_probs.append(batch_probs)

import numpy as np
probs = np.vstack(all_probs)




In [15]:
import numpy as np

# Margin = difference between top two probabilities
sorted_probs = np.sort(probs, axis=1)
margins = sorted_probs[:, -1] - sorted_probs[:, -2]

# Lower margin = higher uncertainty
uncertainty_indices = np.argsort(margins)  # ascending = most uncertain first


## PRENDERE AVERAGE CERTAINTY!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [16]:
K = 100  # or any number you want to label next
uncertain_samples = unlabeled_df.iloc[uncertainty_indices[:K]].copy()

# Save for annotation
uncertain_samples.to_csv("next_batch_to_label.csv", index=False)

In [17]:
# Get predicted label indices
predicted_ids = np.argmax(probs, axis=1)

# Get prediction confidences (max prob per row)
confidences = np.max(probs, axis=1)

# Map back to label names
id2label = {v: k for k, v in label2id.items()}
predicted_labels = [id2label[i] for i in predicted_ids]

# Attach to DataFrame
unlabeled_df["model_prediction"] = predicted_labels
unlabeled_df["model_confidence"] = confidences

## **Label New Batch**

In [18]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os

# 📥 Load the next batch to label
df = pd.read_csv("next_batch_to_label.csv").reset_index(drop=True)

# 📦 Define tone labels
tone_labels = [
    "Neutral / Informational",
    "Supportive / Affirmative / Celebratory",
    "Critical / Angry",
    "Call to Action / Propaganda",
]

# 🧠 Interactive widgets
dropdowns = []
for i, row in df.iterrows():
    tweet_display = widgets.HTML(value=f"<b>Tweet {i+1}</b>: {row['Content']}")
    dropdown = widgets.Dropdown(
        options=[""] + tone_labels,
        description='Tone:',
        layout=widgets.Layout(width='70%')
    )
    dropdowns.append(dropdown)
    display(tweet_display, dropdown)

# 💾 Save button logic — append to existing file
def save_labels(_):
    for i, dropdown in enumerate(dropdowns):
        df.at[i, 'tone_label'] = dropdown.value

    labeled_path = "labeled_tweets_with_tone.csv"

    if os.path.exists(labeled_path):
        existing = pd.read_csv(labeled_path)
        combined = pd.concat([existing, df], ignore_index=True).drop_duplicates(subset=["Content"])
    else:
        combined = df

    combined.to_csv(labeled_path, index=False)
    print(f"✅ Appended {len(df)} tweets. Total labeled: {len(combined)}")

# 🖱️ Save button UI
save_button = widgets.Button(description="💾 Save Labels", button_style='success')
save_button.on_click(save_labels)
display(save_button)


HTML(value='<b>Tweet 1</b>: Fuori dal recinto. Intervista a Nicola Fratoianni\n#governo #sinistra #Draghi #Pd …

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 2</b>: @UnaDonnaPdR @OGiannino @marattin @matteorenzi Poi però mi raccomando. Torna a tuo…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 3</b>: Ho chiesto un incontro urgente al ministro Cartabia, bisogna intervenire subito pe…

...

HTML(value='<b>Tweet 99</b>: @TheG_Ralpha @pontrafab Perché quello di Draghi è un governo istituzionale fatto …

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 100</b>: Nostre sedi e nostri militanti oggetto di intimidazioni intollerabili. Solidarie…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

Button(button_style='success', description='💾 Save Labels', style=ButtonStyle())